In [1]:
from src.data_utils.dataset import build_datset_train_val
from src.Transformers.transformers_prototype import build_transformer_model_v2
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf
from src.constants import TARGET_MAX_LENGHT, MAX_LENGHT_SOURCE

TRIALS=15
EPOCHS=5000
EPOCHS_PER_TRIAL=30

2023-08-21 10:45:08.448887: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 10:45:09.089956: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-21 10:45:09.848628: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-21 10:45:09.867724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
train_dataset, val_dataset = build_datset_train_val(split=0.8, batch_size=128)

train split: 28160 | val split: 6656


In [3]:
def objective(trial):
    tf.keras.backend.clear_session()

    model = build_transformer_model_v2(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_PER_TRIAL, callbacks=get_predefine_callbacks(model_name="v2"))

    levenshtein = model.evaluate(val_dataset)[-1]
    # val_loss = model.evaluate(val_dataset)[0]
    # return  val_loss
    return  levenshtein 

In [4]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=TRIALS, gc_after_trial=True, show_progress_bar=True)

[I 2023-08-21 10:45:10,940] A new study created in memory with name: no-name-35da62d8-1d6b-4982-9447-eac6232a7fa2


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1/30


2023-08-21 10:45:14.687249: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2023-08-21 10:45:15.097044: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-21 10:45:15.284245: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc1c3ed4350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-21 10:45:15.284275: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-08-21 10:45:15.287377: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-21 10:45:15.332375: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-21 10:45:15.368636: I ./tensorflow/compiler/jit/devic

    424/Unknown - 33s 60ms/step - loss: 1.0956 - accuracy: 0.7281 - sparse_levenshtein: inf

...

[I 2023-08-21 13:59:16,673] Trial 13 finished with value: 0.7122923731803894 and parameters: {'attention_heads': 5, 'learning_rate': 0.002607684691624126, 'drop_out_out_decoder': 0.45, 'dense_layers': 2, 'drop_out': 0.30000000000000004, 'encoder_kernel_size': 3}. Best is trial 6 with value: 0.5014358758926392.
Epoch 1/30
    423/Unknown - 38s 75ms/step - loss: 1.1632 - accuracy: 0.7148 - sparse_levenshtein: inf

2023-08-21 13:59:55.254861: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 10629085256694284478
2023-08-21 13:59:55.254887: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 1373443431360807708
2023-08-21 13:59:55.254894: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10413820832664707858
2023-08-21 13:59:55.254899: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3345127408483235702
2023-08-21 13:59:55.254904: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9871231211707845250
2023-08-21 13:59:55.254908: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 7862476319507463156
2023-08-21 13:59:55.254911: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send i

424/424 [==============================] - 41s 84ms/step - loss: 1.1628 - accuracy: 0.7148 - sparse_levenshtein: inf - val_loss: 0.9112 - val_accuracy: 0.7319 - val_sparse_levenshtein: 0.8921
Epoch 2/30
424/424 [==============================] - 35s 82ms/step - loss: 0.8514 - accuracy: 0.7509 - sparse_levenshtein: 0.8203 - val_loss: 0.7869 - val_accuracy: 0.7672 - val_sparse_levenshtein: 0.7741
Epoch 3/30
424/424 [==============================] - 35s 82ms/step - loss: 0.7792 - accuracy: 0.7689 - sparse_levenshtein: 0.7682 - val_loss: 0.7444 - val_accuracy: 0.7775 - val_sparse_levenshtein: 0.7463
Epoch 4/30
424/424 [==============================] - 35s 82ms/step - loss: 0.7461 - accuracy: 0.7768 - sparse_levenshtein: 0.7470 - val_loss: 0.7216 - val_accuracy: 0.7828 - val_sparse_levenshtein: 0.7311
Epoch 5/30
424/424 [==============================] - 35s 82ms/step - loss: 0.7266 - accuracy: 0.7817 - sparse_levenshtein: 0.7327 - val_loss: 0.7076 - val_accuracy: 0.7858 - val_sparse_leve

2023-08-21 14:16:48.826564: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 15813687799482957330
2023-08-21 14:16:48.826586: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 7117018106354482483
2023-08-21 14:16:48.826599: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9579239088160466646
2023-08-21 14:16:48.826605: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 5883691351963938426
2023-08-21 14:16:48.826610: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10300318512929576559
2023-08-21 14:16:48.826616: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 11336483864022638739
2023-08-21 14:16:48.826623: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv 

[I 2023-08-21 14:16:48,840] Trial 14 finished with value: 0.6817783117294312 and parameters: {'attention_heads': 6, 'learning_rate': 0.0018031325935911114, 'drop_out_out_decoder': 0.5, 'dense_layers': 4, 'drop_out': 0.2, 'encoder_kernel_size': 5}. Best is trial 6 with value: 0.5014358758926392.


In [5]:
tf.keras.backend.clear_session()
trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")
    model_name = "v3"

    model = build_transformer_model_v2(trial=trial)
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=get_predefine_callbacks(model_name=model_name))
    print(model.summary())

    print('validation levenshtein distance: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{model_name}")

    print(f"Metrics in Validation: {model.evaluate(val_dataset)}")

Best model: 1
Epoch 1/5000
    424/Unknown - 43s 87ms/step - loss: 1.1069 - accuracy: 0.7258 - sparse_levenshtein: inf

2023-08-21 14:17:32.310973: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10307210475109753058
2023-08-21 14:17:32.310998: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 16860619782365283578
2023-08-21 14:17:32.311029: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 14777190188558647651
2023-08-21 14:17:32.311040: I tensorflow/core/framework/local_rendezvous.cc:409] Local rendezvous send item cancelled. Key hash: 17071032840095908675
2023-08-21 14:17:32.311047: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 14492166920678179973
2023-08-21 14:17:32.311051: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 5521348936875177373
2023-08-21 14:17:32.311055: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous rec

424/424 [==============================] - 47s 96ms/step - loss: 1.1069 - accuracy: 0.7258 - sparse_levenshtein: inf - val_loss: 0.8167 - val_accuracy: 0.7637 - val_sparse_levenshtein: 0.7821
Epoch 2/5000
424/424 [==============================] - 40s 95ms/step - loss: 0.8190 - accuracy: 0.7627 - sparse_levenshtein: 0.7844 - val_loss: 0.7559 - val_accuracy: 0.7767 - val_sparse_levenshtein: 0.7478
Epoch 3/5000
424/424 [==============================] - 40s 95ms/step - loss: 0.7651 - accuracy: 0.7742 - sparse_levenshtein: 0.7532 - val_loss: 0.7223 - val_accuracy: 0.7836 - val_sparse_levenshtein: 0.7281
Epoch 4/5000
424/424 [==============================] - 40s 95ms/step - loss: 0.7346 - accuracy: 0.7819 - sparse_levenshtein: 0.7309 - val_loss: 0.7049 - val_accuracy: 0.7890 - val_sparse_levenshtein: 0.7121
Epoch 5/5000
424/424 [==============================] - 40s 95ms/step - loss: 0.7163 - accuracy: 0.7863 - sparse_levenshtein: 0.7176 - val_loss: 0.6883 - val_accuracy: 0.7914 - val_spa

2023-08-21 14:35:44.355661: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 6543568400596468473
2023-08-21 14:35:44.355680: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 3101791885952818335
2023-08-21 14:35:44.355685: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 13405092000736144755
2023-08-21 14:35:44.355689: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 11961732854413655797
2023-08-21 14:35:44.355692: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9818617059258462161
2023-08-21 14:35:44.355727: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 11458746799526307166
2023-08-21 14:35:44.355743: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv 

In [6]:
# save model

model.save("../models/v3/", save_format="tf")

INFO:tensorflow:Assets written to: ../models/v3/assets


INFO:tensorflow:Assets written to: ../models/v3/assets


In [7]:
from src.data_utils.dataset import char_to_num, num_to_char

In [8]:
target_sequence = [char_to_num[w] for w in ["<"]]

for batch_index, batch in enumerate(val_dataset):
    batch = batch[0]

    sources = batch["source"]
    targets = batch["target"]
    
    print(sources.shape)
    print(targets.shape)

    for index_sample, (source, target) in enumerate(zip(sources, targets)):
        source = tf.expand_dims(source, axis=0)
        target_sequence = [char_to_num[w] for w in ["<"]]
        y_true = "".join([num_to_char[w] for w in target.numpy()])
    
        for i in range(TARGET_MAX_LENGHT):
            next_token = tf.expand_dims(tf.pad(tf.constant(target_sequence), [[0, TARGET_MAX_LENGHT-len(target_sequence)]], mode='CONSTANT', constant_values=0, name=None), axis=0)

            print("next target sequence to predict: ", next_token)

            y_pred = model({"source": source, "target": next_token})

            y_pred = tf.cast(tf.argmax(y_pred, axis=2), dtype=tf.int32)

            print("argmax:", y_pred)

            mask = tf.not_equal(y_pred, 0)
            next_token = y_pred[mask][-1].numpy()

            target_sequence.append(next_token)

            print("sequence so far: ", "".join([num_to_char[w] for w in target_sequence]))
            print("Label: ", y_true)

            if num_to_char[next_token]==">":
                break

        print(f"================================={index_sample}=========================================")
        if index_sample==1:
            break

    if batch_index==1:
        break

(128, 120, 84)
(128, 63)
next target sequence to predict:  tf.Tensor(
[[60  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]], shape=(1, 64), dtype=int32)
argmax: tf.Tensor(
[[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]], shape=(1, 64), dtype=int32)
sequence so far:  <+
Label:  <cointelegraph.com/4158-2/>PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP
next target sequence to predict:  tf.Tensor(
[[60 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]], shape=(1, 64), dtype=int32)
argmax: tf.Tensor(
[[10 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [10]:
from src.custom.metrics import SparseLevenshtein

model_loaded = tf.keras.models.load_model("../models/v3/", custom_objects={"SparseLevenshtein": SparseLevenshtein})

In [11]:
#after register class as serializable
model_loaded = tf.keras.models.load_model("../models/v3/")

# convert to tflite

In [15]:
from src.constants import FEATURE_COLUMNS
from src.data_utils.dataset import start_token_idx, end_token_idx


class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.target_start_token_idx = start_token_idx
        self.target_end_token_idx = end_token_idx
        # Load the feature generation and main models
        self.model = model

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, len(FEATURE_COLUMNS)], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = tf.cast(inputs, tf.float32)
        x = x[None]
        x = tf.cond(tf.shape(x)[1] == 0, lambda: tf.zeros((1, 1, len(FEATURE_COLUMNS))), lambda: tf.identity(x))
        x = x[0]
        x = pre_process(x)
        x = x[None]
        x = self.model.generate(x, self.target_start_token_idx)
        x = x[0]
        idx = tf.argmax(tf.cast(tf.equal(x, self.target_end_token_idx), tf.int32))
        idx = tf.where(tf.math.less(idx, 1), tf.constant(2, dtype=tf.int64), idx)
        x = x[1:idx]
        x = tf.one_hot(x, 59)
        return {'outputs': x}
    
tflitemodel_base = TFLiteModel(model)